In [ ]:
import os
import numpy as np
import itertools
import rasterio
from osgeo import gdal, gdalconst
from joblib import Parallel, delayed

In [ ]:
def generate_list(offset):
    L = list(range(offset+1))
    L1 = [-item for item in L if item != 0]
    L += L1 
    return L
def coord_Nx_Of_NoData(index1,index2,array,buffer):
    idx = list()
    idy = list()    
    if array[index1,index2] == 0:
        for k1 in buffer:
            if index1+k1>=0 and index1+k1<array.shape[0]:      
                for k2 in buffer:
                    if k1 != 0 or k2 != 0:          
                        if index2+k2>=0 and index2+k2<array.shape[1]: 
                            if array[int(index1+k1),int(index2+k2)] == 1:
                                idx.append(int(index1+k1))
                                idy.append(int(index2+k2))
    idx = list(idx)
    idy = list(idy)
    return idx,idy

In [ ]:
bathymetry_dir = r'G:\bathymetry'
original_files_dir = os.path.join(bathymetry_dir,'inputFile')
resampled_files_dir = os.path.join(bathymetry_dir,'resampled')
masked_files_dir = os.path.join(bathymetry_dir,'binaryMask')

tur_dir = r'G:\copernicusTurLayers'
tur_file = 'c_gls_LWQ100-turbidity-blended-mean_201902110000_CUSTOM_MSI_V1.3.tiff'
reference = gdal.Open(os.path.join(tur_dir, tur_file), gdalconst.GA_ReadOnly)
referenceProj = reference.GetProjection()
referenceTrans = reference.GetGeoTransform()
bandreference = reference.GetRasterBand(1)    
x = reference.RasterXSize 
y = reference.RasterYSize

In [ ]:
for fileName in os.listdir(original_files_dir):
    inputFile = gdal.Open(os.path.join(original_files_dir, fileName), gdalconst.GA_ReadOnly)    
    inputProj = inputFile.GetProjection()
    inputTrans = inputFile.GetGeoTransform()
    
    outputFile = os.path.join(resampled_files_dir,'resampled_'+fileName)
    driver= gdal.GetDriverByName('GTiff')
    output = driver.Create(outputFile,x,y,1,bandreference.DataType)
    output.SetGeoTransform(referenceTrans)
    output.SetProjection(referenceProj)
    gdal.ReprojectImage(inputFile,output,inputProj,referenceProj,gdalconst.GRA_NearestNeighbour) 

inputFile.FlushCache()
inputFile = None  
del output

In [ ]:
for fileName in os.listdir(resampled_files_dir):
    inputFile = gdal.Open(os.path.join(resampled_files_dir,fileName))
    inputArray = inputFile.ReadAsArray()
    Mask_Zero = inputArray > -1 
    Ones_array = np.ones((inputArray.shape[0],inputArray.shape[1]),dtype=np.float32)
    Ones_array[Mask_Zero] = 0     
    buffer = generate_list(6)
    coord = Parallel(n_jobs=-1)(delayed(coord_Nx_Of_NoData)(i,j,Ones_array,buffer) for j in range(Ones_array.shape[1]) for i in range(Ones_array.shape[0]))
    coord = [x for x in coord if x != ([], [])] 
    L = []
    for k in range(len(coord)):
        rows = coord[k][0]
        cols = coord[k][1]
        for m in range(len(rows)):
            L.append([rows[m],cols[m]])
    L.sort() 
    L=list(L for L,_ in itertools.groupby(L)) 
    for k in range(len(L)): 
        Ones_array[L[k][0],L[k][1]] = 0 
    Ones_array[80:160,600:790][Ones_array[80:160,600:790]==1]=0 
    subArrays = [Ones_array[50:190,200:400],Ones_array[100:250,520:650]] 
    for sub_array in subArrays:
        buffer = generate_list(8) 
        coord = Parallel(n_jobs=-1)(delayed(coord_Nx_Of_NoData)(i,j,sub_array,buffer) for j in range(sub_array.shape[1]) for i in range(sub_array.shape[0]))
        coord = [x for x in coord if x != ([], [])] 
        L = []
        for k in range(len(coord)):
            rows = coord[k][0]
            cols = coord[k][1]
            for m in range(len(rows)):
                L.append([rows[m],cols[m]])
        L.sort() 
        L=list(L for L,_ in itertools.groupby(L)) 
        for k in range(len(L)): 
            sub_array[L[k][0],L[k][1]] = 0 
    outputFile = os.path.join(masked_files_dir,'binaryMask_'+fileName[14:16]+'.tiff')
    driver= gdal.GetDriverByName('GTiff')    
    output = driver.CreateCopy(outputFile, inputFile) 
    output.GetRasterBand(1).WriteArray(Ones_array)
    output.GetRasterBand(1).SetNoDataValue(0)     
inputFile.FlushCache()
inputFile = None  
del output